In [ ]:
import sys;
sys.path.insert(0, '..')

## Chapter 8 Code Snippets and Listings

### Encoding a number as a frequency using discrete sinc quantum states (section 8.2)

 Listing 8.1 Function to create a circuit for encoding a geometric sequence state with `n` qubits for a given angle `theta`

In [ ]:
from sim_circuit import QuantumRegister, QuantumCircuit

def geometric_sequence_circuit(n, theta):

    N = 2**n

    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for j in range(n): # <1>
        qc.h(q[j])

    for j in range(n): # <1>
        qc.p(2 ** j * theta, q[j])

    return qc

### Phase-to-magnitude frequency encoding with the IQFT (8.2.1)

Listing 8.2 Function to create the circuit for encoding a geometric sequence state with `n` qubits and frequency `v` followed by the IQFT

In [ ]:
from math import pi

def encode_frequency(n, v):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for j in range(n):
        qc.h(q[j])

    for j in range(n):
        qc.p(2 * pi / 2 ** (n - j) * v, q[j]) # <1>

    qc.report('geometric_sequence')

    qc.append_iqft(q)

    qc.report('iqft')

    return qc

### Some useful numerical forms of the frequency encoding pattern (8.2.2)

For example, let's create a three-qubit state with the frequency $v =  4.3$:

In [ ]:
n = 3
v = 4.3
qc = encode_frequency(n, v)
state = qc.run()

Let's check that the magnitudes of the state match the $\vert \text{sincd}_n((v-k) \pi) \vert$ function for $n = 3$ and $v = 4.3$, as defined above.

We will use the function `prod` defined below to compute each product of cosines:

In [ ]:
def prod(iterable):
    p = 1
    for n in iterable:
        p *= n
    return p

We can use the following `assert` statement to check the magnitudes of the example state:

In [ ]:
from math import cos
from util import all_close

N = 2**n
assert all_close([abs(state[k]) for k in range(N)], [
    abs(prod(cos((v - k) * pi / 2 ** (m + 1)) for m in range(n))) for k in
    range(N)])

We can create a phased discrete sinc state using the following function:

In [ ]:
from util import cis

def complex_sincd(n, v):
    N = 2 ** n
    return [prod(
        cos((v - k) * pi / 2 ** (j + 1)) * cis((v - k) * pi / 2 ** (j + 1))
        for j in range(n)) for k in range(2 ** n)]

Let's double-check that the outcome of this function for `n = 3` and `v = 4.3` matches the example state created using the `encode_frequency` function above:

In [ ]:
assert all_close(state, complex_sincd(3, 4.3))

Therefore, the product of complex numbers

$$\prod_{m = 0}^{n-1} \text{cis} \left( (v - k) \frac{\pi}{2^{m + 1}} \right)$$

can also be expressed as

$$\text{cis} \left(\sum_{m = 0}^{n - 1} (v - k) \frac{\pi}{2^{m + 1}} \right) = \text{cis} \left((N - 1) (v - k) \frac{\pi}{N} \right)$$

where $N = 2^n$.

We can use this cis expression, combined with the product of cosines, to create a phased discrete sinc quantum state with the following Python code:

In [ ]:
def complex_sincd_combined_cis(n, v):
    N = 2 ** n
    return [prod(cos((v - k) * pi / 2 ** (m + 1)) for m in range(n)) * cis(
        (N - 1) / N * (v - k) * pi) for k in range(2 ** n)]

Let's check that this form also creates the phased discrete sinc state with `n = 3` and `v = 4.3`:

In [ ]:
assert all_close(state, complex_sincd_combined_cis(3, 4.3))

### An alternative implementation of phased discrete sinc quantum states with reversed qubits (8.2.3)

Listing 8.3 Function to create the circuit for the alternative method for encoding a geometric sequence state with `n` qubits for a given frequency `v`

In [ ]:
def geom_alt(n, v):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for j in range(n):
        qc.h(q[j])

    for j in range(n):
        qc.p(pi * 2 ** -j * v, q[j])

    return qc

In the function above, the angle parameter is defined with $2^{n - j}\theta = \frac{2^n \theta}{2^j} = \frac{2^n}{2^j}\frac{v \pi}{2^n} = \frac{v \pi}{2^j}$

Listing 8.4 Function to create the circuit to encode a frequency in a quantum state using the alternative method

In [ ]:
def encode_frequency_q_alt(n, v):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for j in range(n):
        qc.h(q[j])
        qc.p(pi * 2 ** -j * v, q[j])

    qc.report('signal')

    qc.append_iqft(q, reversed=True, swap=False) #Apply the IQFT to qubit in reverse order and skip the qubit swapping in the IQFT

    qc.report('iqft')

    return qc

### Discrete sinc as a sequence of coin flips (8.3)

We can also model the discrete sinc distribution as a sequence of coin tosses. If the total number of tosses is $n$, the bias of the coin tossed at trial $0 \le m < n$ depends on the previous toss results. If the decimal representation of the binary number formed with the previous flips is $0 \le k < 2^m$, the probabilities of getting 0 or 1 in the $m^{th}$ flip are $\cos^2\left((v-k)\frac{\pi}{2^{m+1}}\right)$ and $\sin^2\left((v-k)\frac{\pi}{2^{m+1}}\right)$, respectively.

In [ ]:
from util import is_close
from math import sin

def discrete_sinc_by_digit(n , v):

    probs = [_ for _ in range(2**n)]
    for l in range(2**n): # <1>
        s = bin(l)[2:].zfill(n)
        assert(len(s) == n)
        p = 1
        k = 0
        for m in range(n): # <2>
            if s[m] == '0':
                p *= cos((v - k)*pi/2**(m+1))**2
            else:
                p *= sin((v - k)*pi/2**(m+1))**2
                k += 2**m

        probs[k] = p

    return probs

Let's validate these probabilities for `n = 3` and `v = 4.7`:

In [ ]:
n = 3
v = 4.7
probs = discrete_sinc_by_digit(n, v)
assert all_close(discrete_sinc_by_digit(n, v), [abs(a)**2 for a in complex_sincd(n, v)])

We can also compute the probabilities for each possible sequence of outcomes with the following recursive function:

In [ ]:
def recursive_discrete_sinc(n, v):
    if n == 1:
        return [cos(v*pi/2)**2, sin(v*pi/2)**2]

    p = recursive_discrete_sinc(n-1, v)

    return [p[k] * cos((v - k) * pi / 2 ** n) ** 2 for k in
            range(2 ** (n - 1))] + [p[k] * sin((v - k) * pi / 2 ** n) ** 2
                                    for k in range(2 ** (n - 1))]

We can also model a sequence of `n` coin tosses with probabilities defined by `n` and the frequency `v` several times and examine the frequency we observe each possible sequence. The function below can be used to simulate a given number (`count`) of `n` coin tosses:

In [ ]:
import numpy as np

def discrete_sinc_coin_flips(n, v, count=10000):
    samples = []
    for _ in range(count):
        k = 0
        for m in range(n):
            flip = np.random.binomial(1, sin((v-k)*pi/2**(m+1))**2)
            k += flip*2**m

        samples.append(k)

    return samples

### Raised cosine (8.4.1)

Listing 8.5 Function for encoding the raised cosine distribution in the probabilities of a quantum state

In [ ]:
def raised_cosine(n, mu):
    N = 2 ** n
    assert (0 <= mu < 2 ** n)

    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.h(q[n - 1])
    qc.p(-pi * mu / N * 2, q[n - 1])

    qc.report('fourier_coefficients')

    qc.append_qft(q, reversed=True, swap=False) # <1>

    qc.report('qft')

    return qc

For example, let's use this function to encode the raised cosine distribution in the probabilities of a three-qubit state with $\mu = 3.25$:

In [ ]:
qc = raised_cosine(3, 3.25)
state = qc.run()

In [ ]:
from util import print_state_table

print_state_table(state)

We can use the following code to check that the amplitudes of the state match the expected amplitudes:

In [ ]:
from math import sqrt

N = 8
mu = 3.25
a = [sqrt(2/N) * cos((k - mu)*pi/N) * cis((k-mu)*pi/N) for k in range(N)]
assert all_close(state, a)

We can also check that the probabilities align with the raised cosine distribution for $s = 2^{n - 2} = \frac{N}{2}$:

In [ ]:
s = N / 2
p = [1 / (2 * s) * (1 + cos((x - mu) / s * pi)) for x in range(N)]
p1 = [1 / s * cos((x - mu) / (2 * s) * pi) ** 2 for x in range(N)]

probs = [2/N*(cos((k - mu)*pi/N))**2 for k in range(N)] 

assert all_close(p, probs)
assert all_close(p1, probs)

### Other trigonometric functions (8.4.2)

We can prepare a state so that the resulting amplitudes will have the probability distribution

$$p(k) = \frac{8}{3N} \sin^4 \left( k \frac{\pi}{N} \right)$$

for $0 \le k < N$. We call this the $sin^4$ probability distribution.

Listing 8.6 Function for encoding the $sin^4$ probability distribution in a quantum state

In [ ]:
from math import acos

def sin_4(n):
    theta = acos(sqrt(2 / 3))
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.ry(2 * theta, q[n - 1])
    qc.p(pi, q[n - 1])
    qc.cry(pi / 2, q[n - 1], q[0])

    for i in range(1, n - 1):
        qc.cx(q[0], q[i])

    qc.report('frequencies')

    qc.append_qft(q, reversed=True, swap=False)

    qc.report('qft')

    return qc

Let's create an example with `n = 3` qubits:

In [ ]:
n = 3           
N = 2 ** n      
qc = sin_4(n)   
state = qc.run()

In [ ]:
print_state_table(state)

We can check that the probabilities of the resulting state reflect the encoded probability distribution using the following code:

In [ ]:
s = [sqrt(8 / (3 * N)) * (sin(k * pi / N)) ** 2 for k in range(N)]
assert all_close(state, s)
p = [8 / 3 / N * (sin(k * pi / N)) ** 4 for k in range(N)] # <1>
assert all_close([abs(state[k])**2 for k in range(N)], p)